In [ ]:
!pip install -q transformers==4.46.3 datasets peft accelerate bitsandbytes sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 93.4 MB/s eta 0:00:00


In [ ]:
import torch
import json
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)

# print("Libraries installed. Checking GPU...")
# if not torch.cuda.is_available():
#     raise RuntimeError("GPU not found! Go to Runtime > Change runtime type > T4 GPU.")
# print(f"Using GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Path to your dataset
DATA_PATH = "/content/drive/MyDrive/datasets/ready_to_train.json" #@param {type:"string"}


In [ ]:
# ==========================================
# 2. CONFIGURATION (Optimized for 1 Hour)
# ==========================================
# MODEL_ID = "google/flan-t5-small"  #
MODEL_ID = "google/flan-t5-base"
# MODEL_ID = "google/flan-t5-large"
# DATA_PATH = "ready_to_train.json" # Your uploaded file
OUTPUT_DIR = "./t5-natsql-lora"

# Training speed params`
MAX_SOURCE_LEN = 512
MAX_TARGET_LEN = 128
LEARNING_RATE = 1e-3  # High LR for LoRA
EPOCHS = 5            # Start with 3. If fast, we can do more.
BATCH_SIZE = 8        # Physical batch size per GPU
GRAD_ACCUM = 4        # Effective batch size = 32

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [ ]:
# 2. Preprocess (Standard T5 setup)
def preprocess_function(examples):
    inputs = [f"Translate to NatSQL: {q} | {s}" for q, s in zip(examples['question'], examples['schema_serialization'])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Setup targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['natsql'], max_length=128, truncation=True, padding="max_length")

    # Replace padding with -100 for ignore_index
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Reload data just to be safe
raw_dataset = load_dataset('json', data_files=DATA_PATH, split='train')
# 2. Split FIRST (Creates a DatasetDict with 'train' and 'test' keys)
split_datasets = raw_dataset.train_test_split(test_size=0.1, seed=42)
tokenized_datasets = split_datasets.map(preprocess_function, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/6300 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

In [ ]:
# # Map over dataset
# print(f"Tokenizing {len(dataset)} examples...")
# tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)
# print("Data ready.")

In [ ]:
# # Get the first processed example
# example_labels = tokenized_dataset[0]['labels']

# # Check how many -100s are there
# ignored_count = example_labels.count(-100)
# total_count = len(example_labels)

# print(f"Total tokens: {total_count}")
# print(f"Ignored tokens (-100): {ignored_count}")
# print(f"First 10 tokens: {example_labels[:10]}")

# if ignored_count == total_count:
#     print("CRITICAL ERROR: The entire label is being ignored!")
# else:
#     print("Data looks okay... investigating Learning Rate.")

In [ ]:
# ==========================================
# 4. MODEL & LORA SETUP
# ==========================================
print("Loading Model...")
# Load in FP32 initially, trainer will handle mixed precision
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)

Loading Model...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# # LoRA Configuration
# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_2_SEQ_LM,
#     inference_mode=False,
#     r=16,               # Rank 16
#     lora_alpha=32,      # Scaling factor
#     lora_dropout=0.1,
#     target_modules=["q", "v"] # Adapt Attention layers
# )
# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1, target_modules=["q", "v"]
# )

In [ ]:
# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()
# Expect: trainable params: ~1-2%

# model = get_peft_model(model, peft_config)

In [ ]:
# 5. TRAINING
# ==========================================
# 3. Training Arguments (Standard Full Fine-Tuning)
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-spider-robust",
    learning_rate=3e-4,          # Standard LR for full fine-tuning
    per_device_train_batch_size=8, # t5-small is tiny, we can use bigger batches
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="no",
    fp16=False,                   # FP16 works fine on standard models
    bf16=False,
    optim="adamw_torch",
    report_to="none",
    max_grad_norm=1.0,
    evaluation_strategy="steps",
    save_total_limit=2,
)

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'], # The 90%
    eval_dataset=tokenized_datasets['test'],   # The 10%
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
)

/tmp/ipython-input-419926589.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
print("Starting Training...")
trainer.train()

Starting Training...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
10,2.033600,0.748244
20,0.829900,0.472148
30,0.592500,0.369805
40,0.512200,0.306723
50,0.409700,0.272140
60,0.405900,0.247623
70,0.358100,0.223972
80,0.360200,0.211231
90,0.302500,0.195457
100,0.294700,0.188807


TrainOutput(global_step=591, training_loss=0.22271288417014978, metrics={'train_runtime': 5369.9936, 'train_samples_per_second': 3.52, 'train_steps_per_second': 0.11, 'total_flos': 1.29419139612672e+16, 'train_loss': 0.22271288417014978, 'epoch': 3.0})

In [ ]:
print("Saving fine-tuned model...")
model_output_dir = "./t5-natsql-finetuned"

# Save the fine-tuned model directly (no merge_and_unload needed for full fine-tuning)
model.save_pretrained(model_output_dir)

# Also save the tokenizer
tokenizer.save_pretrained(model_output_dir)

print(f"DONE! Full fine-tuned model saved to folder '{model_output_dir}'")

Saving fine-tuned model...
DONE! Full fine-tuned model saved to folder './t5-natsql-finetuned'


In [ ]:
from google.colab import drive
import shutil
import os

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define source (where the model is now) and destination (Drive)
source_folder = "./t5-natsql-finetuned"
destination_folder = "/content/drive/MyDrive/Text-to-SQL-Project/t5-natsql-adapter-final2"

# 3. Check if destination exists, if not create it
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder, exist_ok=True)
    print(f"Created folder: {destination_folder}")

# 4. Copy the files
print(f"Copying model from {source_folder} to Google Drive...")
# We use copytree if dir doesn't exist, or copy files individually if it does
# Simple approach: delete destination if exists to ensure fresh copy, then copy
if os.path.exists(destination_folder):
    shutil.rmtree(destination_folder)
shutil.copytree(source_folder, destination_folder)

print(f"✅ SUCCESS! Model saved to: {destination_folder}")
print("You can now safely close this notebook.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Created folder: /content/drive/MyDrive/Text-to-SQL-Project/t5-natsql-adapter-final2
Copying model from ./t5-natsql-finetuned to Google Drive...
✅ SUCCESS! Model saved to: /content/drive/MyDrive/Text-to-SQL-Project/t5-natsql-adapter-final2
You can now safely close this notebook.


In [ ]:
from huggingface_hub import login

# Paste your token when asked (it won't show on screen, just paste and press Enter)
login()

In [ ]:

repo_id = "blacknahil/t5-base-spider-v1-finetuned"

print(f"Pushing to {repo_id}...")

# 2. Push the Model
model.push_to_hub(
    repo_id,
    commit_message="Training complete: T5-Base on Spider (10 Epochs)"
)

# 3. Push the Tokenizer (Crucial for others to use it)
tokenizer.push_to_hub(repo_id)

print("✅ Upload complete!")

Pushing to blacknahil/t5-base-spider-v1-finetuned...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...euwst4c/model.safetensors:   0%|          |  554kB /  990MB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  .../tmp1q8w5ert/spiece.model:  86%|########5 |  679kB /  792kB            

✅ Upload complete!


In [ ]:
from huggingface_hub import HfApi

# 1. Write your description
readme_text = f"""---
language: en
tags:
- text-to-sql
- spider
- t5
datasets:
- spider
widget:
- text: "Translate to SQL: How many singers are there? | schema: singer : singer_id , name , country"
---

# T5-Base Spider (Robust Version)

This model is fine-tuned on the **Spider** dataset to translate natural language questions into SQL queries.

## Model Details
- **Base Model:** google/flan-t5-base
- **Dataset:** Spider (Text-to-SQL)
- **Training Epochs:** 10
- **Training Time:** ~4 Hours

## How to Use
```python
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("{repo_id}")
model = AutoModelForSeq2SeqLM.from_pretrained("{repo_id}")

input_text = "Translate to SQL: Show all students who are older than 20 | schema: student : id , name , age"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
"""

In [ ]:
with open("README.md", "w") as f:
  f.write(readme_text)

In [ ]:
api = HfApi()
api.upload_file( path_or_fileobj="README.md", path_in_repo="README.md", repo_id=repo_id, repo_type="model" )

CommitInfo(commit_url='https://huggingface.co/blacknahil/t5-base-spider-v1-finetuned/commit/03d12f562e0dfeda97e97965bb17f7319ee344cf', commit_message='Upload README.md with huggingface_hub', commit_description='', oid='03d12f562e0dfeda97e97965bb17f7319ee344cf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/blacknahil/t5-base-spider-v1-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='blacknahil/t5-base-spider-v1-finetuned'), pr_revision=None, pr_num=None)